In [34]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
headers={"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_2) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.0.2 Safari/605.1.15"
        }

In [3]:
icd_chap_url=u'http://code.nhsa.gov.cn:8000/jbzd/public/toStdIcdTreeList.html'
icd_chapter=requests.get(icd_chap_url,headers=headers)

In [5]:
icd_chapter=icd_chapter.json()

In [8]:
icd_chapter[0]['icdId']

'f5140552-795f-11e9-9611-8cec4bd010f3'

In [9]:
icd_detail_url_base=u'http://code.nhsa.gov.cn:8000/jbzd/public/dataWesterSearchDetail.html?icdId='

In [10]:
icd_detail_url_full=icd_detail_url_base+icd_chapter[0]['icdId']

In [11]:
icd_detail_url_full

'http://code.nhsa.gov.cn:8000/jbzd/public/dataWesterSearchDetail.html?icdId=f5140552-795f-11e9-9611-8cec4bd010f3'

In [21]:
icd_detail=requests.get(icd_detail_url_full,headers=headers)

In [31]:
if icd_detail.status_code == 200:
    info = {}
    soup = BeautifulSoup(icd_detail.text)
    icd_code_lst=[d.text.strip() for d in soup.find_all('a')]
    icd_name_lst=[d.text.strip() for d in soup.find_all('span')]


In [35]:
CHS_ICD10=[]
for i in range(len(icd_chapter)):
    icd_detail_url_full=icd_detail_url_base+icd_chapter[i]['icdId']
    icd_detail=requests.get(icd_detail_url_full,headers=headers)
    if icd_detail.status_code == 200:
        info = {}
        soup = BeautifulSoup(icd_detail.text)
        icd_code_lst=[d.text.strip() for d in soup.find_all('a')]
        icd_name_lst=[d.text.strip() for d in soup.find_all('span')]
        df_icd=pd.DataFrame({'icd_code':icd_code_lst,'icd_name':icd_name_lst})
        CHS_ICD10.append(df_icd)
    else:
        print(icd_detail_url_full)

In [37]:
CHS_ICD10_Table=pd.concat(CHS_ICD10,axis=0,ignore_index=True)

In [39]:
CHS_ICD10_Table.head(20)

,icd_code,icd_name
0,A00,霍乱
1,A00.0,霍乱，由于O1群霍乱弧菌，霍乱生物型所致
2,A00.000,霍乱，由于O1群霍乱弧菌，霍乱生物型所致
3,A00.000x001,古典生物型霍乱
4,A00.1,霍乱，由于O1群霍乱弧菌，埃尔托生物型所致
5,A00.100,霍乱，由于O1群霍乱弧菌，埃尔托生物型所致
6,A00.100x001,埃尔托生物型霍乱
7,A00.9,未特指的霍乱
8,A00.900,霍乱
9,A00.900x002,霍乱轻型


In [40]:
len(CHS_ICD10_Table)

47101

In [42]:
CHS_ICD10_Table.to_csv('./LOOKUP_TABLES/CHS_ICD10.csv',encoding='utf-8',index=False)